<a href="https://colab.research.google.com/github/Nastarii/PCB-defect-detection/blob/28ago/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install imageio

### Libraries

---



In [1]:
import tensorflow
import cv2 as cv
import pandas as pd
import os
import matplotlib as plt
import PIL
import torch
import imgaug
import imageio